### La idea es que empecemos en este File a procesar la data

In [1]:
using CSV
using DataFrames
using StatsBase

In [2]:
using Plots
using ImageView
using FileIO

In [10]:
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON
using ColorVectorSpace

In [63]:
using MLDataUtils

In [69]:
using Flux: @epochs

In [4]:
#Se crea un dataframe en donde la primer columna es el id de la imagen y la segunda el label de si la imagen posee cancer o no
# label: 0 no tiene, 1 tiene
df = DataFrame(CSV.File("./train_labels.csv"))

Row,id,label
,String,Int64
1,f38a6374c348f90b587e046aac6079959adf3835,0
2,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
3,755db6279dae599ebb4d39a9123cce439965282d,0
4,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
5,068aba587a4950175d04c680d38943fd488d6a9d,0
6,acfe80838488fae3c89bd21ade75be5c34e66be7,0
7,a24ce148f6ffa7ef8eefb4efb12ebffe8dd700da,1
8,7f6ccae485af121e0b6ee733022e226ee6b0c65f,1
9,559e55a64c9ba828f700e948f6886f4cea919261,0


In [5]:
value_counts(df, col) = combine(groupby(df, col), nrow)

value_counts (generic function with 1 method)

In [6]:
value_counts(df, "label")

Row,label,nrow
,Int64,Int64
1,0,130908
2,1,89117


In [7]:
#Nos quedamos con 1000 filas para que el procesamiento sea mas rápido, de manera aleatoria
df = df[sample(axes(df, 1), 1000; replace = false, ordered = true), :]

Row,id,label
,String,Int64
1,0655891cc0f8cff190a182c0511ce12f5d9e0145,1
2,50bd266a907e0a5c648e959277745ddde0b88993,0
3,f1be09086e94f056d0089dc0f9e812cf65dcbed7,0
4,e91d656c816edc5c1931c61699c87552e0914adc,0
5,08053e213515a9fffd813ba52e79caefd816f8a3,1
6,325844041d0e9f2a7cde0b2a2c05128175a240ff,0
7,a3ae00594011f19aeec86c41d739c0f000981cb6,0
8,4984042a48026f8facf3b58fdb33ea5f6b562526,0
9,21688ea4804321d49a7a24cd34567e0c67e1b31f,1


In [8]:
# Agarramos un sample de 5 imágenes con cancer positivo
es_positivo(valor::Integer) = valor == 1
positivos = filter(:label => es_positivo, df)
positivos_sample = positivos[sample(axes(positivos, 1), 5; replace = false, ordered = true), :]

Row,id,label
,String,Int64
1,f9b18029b15496528f6789c72e3d80d361ae16e7,1
2,e0a3cf4aa5f586e7b6de495aa62766aab20b7c5b,1
3,4363bdd4352fdaf85eb234c3bbc1bb8aaa4516ea,1
4,bd2e7755d4479bb37da51e777553a3e2f4df712f,1
5,b846e148957c1ec8d0c33f5de684c93639695791,1


In [9]:
# Agarramos un sample de 5 imágenes con cancer negativo
es_negativo(valor::Integer) = valor == 0
negativos = filter(:label => es_negativo, df)
negativos_sample = negativos[sample(axes(negativos, 1), 5; replace = false, ordered = true), :]

Row,id,label
,String,Int64
1,097b7d880b8e08bb52446ecc226f1704b5745e61,0
2,f63bb714b8b95b5937f5e45515366db6339ffbad,0
3,74e347058063c1c49064cf9f49745f25f2603000,0
4,60c975f9c794f75076804c15adcf88a3ec4e14c2,0
5,a0b25fea792c1b48996dc9019920cdabf2aa6ea8,0


In [11]:
# Creamos una nueva columna con el path de cada imagen 
df.image_path = string.("../input/histopathologic-cancer-detection/train/", df.id, ".tif")

1000-element Vector{String}:
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "ff190a182c0511ce12f5d9e0145.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "a5c648e959277745ddde0b88993.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "056d0089dc0f9e812cf65dcbed7.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "c5c1931c61699c87552e0914adc.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "9fffd813ba52e79caefd816f8a3.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "f2a7cde0b2a2c05128175a240ff.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "19aeec86c41d739c0f000981cb6.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "f8facf3b58fdb33ea5f6b562526.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "1d49a7a24cd34567e0c67e1b31f.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "8bb52446ecc226f1704b5745e61.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "19ecf8e29a6fa09acfaba064525.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "700936083591d58d4f9e

In [12]:
# Creamos una lista de tipo matriz de RGB donde se encuentran las componentes RGB de cada pixel de la imagen
# Para ello se recorre cada fila del dataframe y con el path de la imagen y la función load() se convierte la imagen a 
# matriz RGB
train_images = Matrix{RGB{}}[]
for row in eachrow(df)
    push!(train_images, load(row.image_path))
end

In [13]:
train_images

1000-element Vector{Matrix{RGB}}:
 [RGB{N0f8}(0.227,0.145,0.549) RGB{N0f8}(0.267,0.169,0.569) … RGB{N0f8}(0.153,0.063,0.329) RGB{N0f8}(0.129,0.055,0.306); RGB{N0f8}(0.173,0.024,0.427) RGB{N0f8}(0.275,0.106,0.518) … RGB{N0f8}(0.149,0.043,0.322) RGB{N0f8}(0.122,0.024,0.29); … ; RGB{N0f8}(0.655,0.341,0.714) RGB{N0f8}(0.573,0.255,0.627) … RGB{N0f8}(0.251,0.078,0.373) RGB{N0f8}(0.251,0.078,0.373); RGB{N0f8}(0.392,0.098,0.471) RGB{N0f8}(0.612,0.298,0.678) … RGB{N0f8}(0.231,0.059,0.353) RGB{N0f8}(0.204,0.031,0.325)]
 [RGB{N0f8}(0.89,0.71,0.812) RGB{N0f8}(0.882,0.718,0.812) … RGB{N0f8}(0.518,0.396,0.455) RGB{N0f8}(1.0,0.91,0.969); RGB{N0f8}(0.957,0.78,0.871) RGB{N0f8}(0.914,0.749,0.835) … RGB{N0f8}(0.91,0.8,0.855) RGB{N0f8}(0.796,0.686,0.741); … ; RGB{N0f8}(0.957,0.749,0.812) RGB{N0f8}(0.714,0.506,0.576) … RGB{N0f8}(1.0,0.949,1.0) RGB{N0f8}(0.914,0.827,0.878); RGB{N0f8}(0.988,0.8,0.855) RGB{N0f8}(0.847,0.659,0.714) … RGB{N0f8}(0.792,0.702,0.765) RGB{N0f8}(0.839,0.745,0.808)]
 [RGB{N0f8}(0.396,

In [17]:
# Insertamos la nueva columna con la información RGB correspondiente a cada imagen
df.images = train_images

1000-element Vector{Matrix{RGB}}:
 [RGB{N0f8}(0.227,0.145,0.549) RGB{N0f8}(0.267,0.169,0.569) … RGB{N0f8}(0.153,0.063,0.329) RGB{N0f8}(0.129,0.055,0.306); RGB{N0f8}(0.173,0.024,0.427) RGB{N0f8}(0.275,0.106,0.518) … RGB{N0f8}(0.149,0.043,0.322) RGB{N0f8}(0.122,0.024,0.29); … ; RGB{N0f8}(0.655,0.341,0.714) RGB{N0f8}(0.573,0.255,0.627) … RGB{N0f8}(0.251,0.078,0.373) RGB{N0f8}(0.251,0.078,0.373); RGB{N0f8}(0.392,0.098,0.471) RGB{N0f8}(0.612,0.298,0.678) … RGB{N0f8}(0.231,0.059,0.353) RGB{N0f8}(0.204,0.031,0.325)]
 [RGB{N0f8}(0.89,0.71,0.812) RGB{N0f8}(0.882,0.718,0.812) … RGB{N0f8}(0.518,0.396,0.455) RGB{N0f8}(1.0,0.91,0.969); RGB{N0f8}(0.957,0.78,0.871) RGB{N0f8}(0.914,0.749,0.835) … RGB{N0f8}(0.91,0.8,0.855) RGB{N0f8}(0.796,0.686,0.741); … ; RGB{N0f8}(0.957,0.749,0.812) RGB{N0f8}(0.714,0.506,0.576) … RGB{N0f8}(1.0,0.949,1.0) RGB{N0f8}(0.914,0.827,0.878); RGB{N0f8}(0.988,0.8,0.855) RGB{N0f8}(0.847,0.659,0.714) … RGB{N0f8}(0.792,0.702,0.765) RGB{N0f8}(0.839,0.745,0.808)]
 [RGB{N0f8}(0.396,

In [18]:
df

Row,id,label,image_path,images
,String,Int64,String,Array…
1,0655891cc0f8cff190a182c0511ce12f5d9e0145,1,../input/histopathologic-cancer-detection/train/0655891cc0f8cff190a182c0511ce12f5d9e0145.tif,"RGB[RGB{N0f8}(0.227,0.145,0.549) RGB{N0f8}(0.267,0.169,0.569) … RGB{N0f8}(0.153,0.063,0.329) RGB{N0f8}(0.129,0.055,0.306); RGB{N0f8}(0.173,0.024,0.427) RGB{N0f8}(0.275,0.106,0.518) … RGB{N0f8}(0.149,0.043,0.322) RGB{N0f8}(0.122,0.024,0.29); … ; RGB{N0f8}(0.655,0.341,0.714) RGB{N0f8}(0.573,0.255,0.627) … RGB{N0f8}(0.251,0.078,0.373) RGB{N0f8}(0.251,0.078,0.373); RGB{N0f8}(0.392,0.098,0.471) RGB{N0f8}(0.612,0.298,0.678) … RGB{N0f8}(0.231,0.059,0.353) RGB{N0f8}(0.204,0.031,0.325)]"
2,50bd266a907e0a5c648e959277745ddde0b88993,0,../input/histopathologic-cancer-detection/train/50bd266a907e0a5c648e959277745ddde0b88993.tif,"RGB[RGB{N0f8}(0.89,0.71,0.812) RGB{N0f8}(0.882,0.718,0.812) … RGB{N0f8}(0.518,0.396,0.455) RGB{N0f8}(1.0,0.91,0.969); RGB{N0f8}(0.957,0.78,0.871) RGB{N0f8}(0.914,0.749,0.835) … RGB{N0f8}(0.91,0.8,0.855) RGB{N0f8}(0.796,0.686,0.741); … ; RGB{N0f8}(0.957,0.749,0.812) RGB{N0f8}(0.714,0.506,0.576) … RGB{N0f8}(1.0,0.949,1.0) RGB{N0f8}(0.914,0.827,0.878); RGB{N0f8}(0.988,0.8,0.855) RGB{N0f8}(0.847,0.659,0.714) … RGB{N0f8}(0.792,0.702,0.765) RGB{N0f8}(0.839,0.745,0.808)]"
3,f1be09086e94f056d0089dc0f9e812cf65dcbed7,0,../input/histopathologic-cancer-detection/train/f1be09086e94f056d0089dc0f9e812cf65dcbed7.tif,"RGB[RGB{N0f8}(0.396,0.2,0.431) RGB{N0f8}(0.745,0.561,0.773) … RGB{N0f8}(0.765,0.616,0.667) RGB{N0f8}(0.655,0.502,0.576); RGB{N0f8}(0.686,0.518,0.698) RGB{N0f8}(0.337,0.169,0.345) … RGB{N0f8}(0.918,0.784,0.847) RGB{N0f8}(0.839,0.702,0.788); … ; RGB{N0f8}(0.992,0.898,0.953) RGB{N0f8}(0.914,0.827,0.878) … RGB{N0f8}(0.984,0.882,0.925) RGB{N0f8}(0.953,0.855,0.839); RGB{N0f8}(0.89,0.808,0.875) RGB{N0f8}(0.882,0.808,0.882) … RGB{N0f8}(1.0,0.945,0.976) RGB{N0f8}(0.894,0.808,0.765)]"
4,e91d656c816edc5c1931c61699c87552e0914adc,0,../input/histopathologic-cancer-detection/train/e91d656c816edc5c1931c61699c87552e0914adc.tif,"RGB[RGB{N0f8}(0.898,0.761,0.918) RGB{N0f8}(0.82,0.667,0.843) … RGB{N0f8}(1.0,0.949,1.0) RGB{N0f8}(0.875,0.812,0.914); RGB{N0f8}(0.89,0.757,0.855) RGB{N0f8}(0.729,0.576,0.698) … RGB{N0f8}(0.89,0.812,0.847) RGB{N0f8}(0.624,0.549,0.627); … ; RGB{N0f8}(0.894,0.725,0.843) RGB{N0f8}(0.745,0.584,0.718) … RGB{N0f8}(0.808,0.631,0.761) RGB{N0f8}(0.8,0.612,0.769); RGB{N0f8}(0.694,0.553,0.655) RGB{N0f8}(0.843,0.698,0.82) … RGB{N0f8}(0.847,0.667,0.769) RGB{N0f8}(0.855,0.671,0.796)]"
5,08053e213515a9fffd813ba52e79caefd816f8a3,1,../input/histopathologic-cancer-detection/train/08053e213515a9fffd813ba52e79caefd816f8a3.tif,"RGB[RGB{N0f8}(0.89,0.651,0.812) RGB{N0f8}(1.0,0.831,0.969) … RGB{N0f8}(0.82,0.631,0.733) RGB{N0f8}(0.867,0.69,0.773); RGB{N0f8}(0.502,0.263,0.427) RGB{N0f8}(0.506,0.333,0.475) … RGB{N0f8}(1.0,0.859,0.937) RGB{N0f8}(0.749,0.576,0.635); … ; RGB{N0f8}(0.275,0.086,0.282) RGB{N0f8}(0.263,0.11,0.282) … RGB{N0f8}(0.933,0.812,0.831) RGB{N0f8}(1.0,0.969,0.973); RGB{N0f8}(0.271,0.086,0.298) RGB{N0f8}(0.235,0.082,0.259) … RGB{N0f8}(0.855,0.753,0.757) RGB{N0f8}(1.0,0.918,0.906)]"
6,325844041d0e9f2a7cde0b2a2c05128175a240ff,0,../input/histopathologic-cancer-detection/train/325844041d0e9f2a7cde0b2a2c05128175a240ff.tif,"RGB[RGB{N0f8}(0.31,0.161,0.596) RGB{N0f8}(0.329,0.204,0.584) … RGB{N0f8}(0.647,0.412,0.757) RGB{N0f8}(0.38,0.114,0.455); RGB{N0f8}(0.235,0.094,0.486) RGB{N0f8}(0.282,0.157,0.537) … RGB{N0f8}(0.871,0.659,0.988) RGB{N0f8}(0.545,0.294,0.624); … ; RGB{N0f8}(0.624,0.361,0.741) RGB{N0f8}(0.89,0.643,0.918) … RGB{N0f8}(0.561,0.38,0.808) RGB{N0f8}(0.451,0.29,0.651); RGB{N0f8}(0.918,0.627,1.0) RGB{N0f8}(0.965,0.706,0.957) … RGB{N0f8}(0.408,0.235,0.686) RGB{N0f8}(0.596,0.435,0.796)]"
7,a3ae00594011f19aeec86c41d739c0f000981cb6,0,../input/histopathologic-cancer-detection/train/a3ae00594011f19aeec86c41d739c0f000981cb6.tif,"RGB[RGB{N0f8}(0.863,0.561,0.78) RGB{N0f8}(0.729,0.455,0.651) … RGB{N0f8}(0.804,0.71,0.765) RGB{N0f8}(0.867,0

In [23]:
df_train = first(df, 700)
df_valid = last(df, 300)

Row,id,label,image_path,images
,String,Int64,String,Array…
1,c0062ff9cca635f9437a525c54848b444e302532,0,../input/histopathologic-cancer-detection/train/c0062ff9cca635f9437a525c54848b444e302532.tif,"RGB[RGB{N0f8}(0.624,0.494,0.561) RGB{N0f8}(0.212,0.075,0.161) … RGB{N0f8}(0.31,0.145,0.294) RGB{N0f8}(0.42,0.216,0.365); RGB{N0f8}(0.875,0.643,0.761) RGB{N0f8}(0.608,0.392,0.518) … RGB{N0f8}(0.325,0.149,0.302) RGB{N0f8}(0.494,0.29,0.439); … ; RGB{N0f8}(0.196,0.106,0.216) RGB{N0f8}(0.192,0.082,0.231) … RGB{N0f8}(0.624,0.447,0.6) RGB{N0f8}(0.125,0.0,0.133); RGB{N0f8}(0.388,0.318,0.412) RGB{N0f8}(0.235,0.141,0.275) … RGB{N0f8}(0.365,0.216,0.376) RGB{N0f8}(0.149,0.039,0.2)]"
2,28616ce24292c1727522b35f10c4def61c3f962f,0,../input/histopathologic-cancer-detection/train/28616ce24292c1727522b35f10c4def61c3f962f.tif,"RGB[RGB{N0f8}(0.396,0.11,0.533) RGB{N0f8}(0.369,0.153,0.553) … RGB{N0f8}(0.514,0.259,0.659) RGB{N0f8}(0.749,0.455,0.867); RGB{N0f8}(0.42,0.2,0.596) RGB{N0f8}(0.384,0.212,0.592) … RGB{N0f8}(0.549,0.353,0.733) RGB{N0f8}(0.4,0.157,0.553); … ; RGB{N0f8}(0.267,0.09,0.447) RGB{N0f8}(0.38,0.188,0.549) … RGB{N0f8}(0.702,0.392,0.753) RGB{N0f8}(0.651,0.302,0.631); RGB{N0f8}(0.325,0.165,0.525) RGB{N0f8}(0.659,0.482,0.847) … RGB{N0f8}(0.792,0.482,0.839) RGB{N0f8}(0.71,0.361,0.69)]"
3,7c94f1592863be0a707834c45f694f5e730267bb,0,../input/histopathologic-cancer-detection/train/7c94f1592863be0a707834c45f694f5e730267bb.tif,"RGB[RGB{N0f8}(0.576,0.271,0.459) RGB{N0f8}(0.553,0.286,0.471) … RGB{N0f8}(0.4,0.196,0.388) RGB{N0f8}(0.376,0.169,0.38); RGB{N0f8}(0.729,0.455,0.6) RGB{N0f8}(0.698,0.455,0.6) … RGB{N0f8}(0.416,0.227,0.424) RGB{N0f8}(0.31,0.122,0.318); … ; RGB{N0f8}(0.882,0.765,0.733) RGB{N0f8}(0.976,0.847,0.882) … RGB{N0f8}(0.792,0.584,0.655) RGB{N0f8}(0.882,0.675,0.737); RGB{N0f8}(0.992,0.859,0.812) RGB{N0f8}(1.0,0.949,0.992) … RGB{N0f8}(0.784,0.584,0.651) RGB{N0f8}(0.706,0.502,0.553)]"
4,4cee0d4666424cef1ea1e0412363edb4737c27d1,0,../input/histopathologic-cancer-detection/train/4cee0d4666424cef1ea1e0412363edb4737c27d1.tif,"RGB[RGB{N0f8}(0.894,0.886,0.89) RGB{N0f8}(0.902,0.886,0.89) … RGB{N0f8}(1.0,0.973,1.0) RGB{N0f8}(0.753,0.69,0.729); RGB{N0f8}(0.898,0.898,0.898) RGB{N0f8}(0.902,0.894,0.898) … RGB{N0f8}(0.933,0.851,0.91) RGB{N0f8}(1.0,0.973,1.0); … ; RGB{N0f8}(0.976,0.965,1.0) RGB{N0f8}(0.894,0.89,0.914) … RGB{N0f8}(0.878,0.859,0.886) RGB{N0f8}(0.871,0.851,0.878); RGB{N0f8}(1.0,0.992,1.0) RGB{N0f8}(0.886,0.882,0.906) … RGB{N0f8}(0.922,0.902,0.929) RGB{N0f8}(0.922,0.902,0.929)]"
5,68788b461d1c31a6175e701695278892dde0745f,0,../input/histopathologic-cancer-detection/train/68788b461d1c31a6175e701695278892dde0745f.tif,"RGB[RGB{N0f8}(0.878,0.871,0.89) RGB{N0f8}(0.878,0.871,0.89) … RGB{N0f8}(0.875,0.867,0.886) RGB{N0f8}(0.875,0.867,0.886); RGB{N0f8}(0.878,0.871,0.89) RGB{N0f8}(0.878,0.871,0.89) … RGB{N0f8}(0.875,0.867,0.886) RGB{N0f8}(0.875,0.867,0.886); … ; RGB{N0f8}(0.651,0.447,0.627) RGB{N0f8}(0.651,0.467,0.631) … RGB{N0f8}(0.875,0.863,0.89) RGB{N0f8}(0.875,0.863,0.89); RGB{N0f8}(0.722,0.502,0.686) RGB{N0f8}(0.835,0.643,0.82) … RGB{N0f8}(0.867,0.855,0.882) RGB{N0f8}(0.867,0.855,0.882)]"
6,4e6000b6504ea7bc2ad6a9c7b6644a7b0b62a70d,0,../input/histopathologic-cancer-detection/train/4e6000b6504ea7bc2ad6a9c7b6644a7b0b62a70d.tif,"RGB[RGB{N0f8}(0.953,0.792,0.855) RGB{N0f8}(0.918,0.757,0.82) … RGB{N0f8}(0.925,0.635,0.741) RGB{N0f8}(0.886,0.596,0.702); RGB{N0f8}(0.839,0.675,0.729) RGB{N0f8}(0.878,0.714,0.769) … RGB{N0f8}(0.929,0.694,0.792) RGB{N0f8}(1.0,0.784,0.882); … ; RGB{N0f8}(0.929,0.78,0.863) RGB{N0f8}(0.663,0.518,0.588) … RGB{N0f8}(0.753,0.608,0.678) RGB{N0f8}(0.839,0.69,0.78); RGB{N0f8}(0.863,0.725,0.753) RGB{N0f8}(0.878,0.753,0.773) … RGB{N0f8}(0.969,0.812,0.918) RGB{N0f8}(0.706,0.545,0.663)]"
7,7f068543d52ac5604c4cf64d2f379c5c61086801,1,../input/histopathologic-cancer-detection/train/7f068543d52ac5604c4cf64d2f379c5c61086801.tif,"RGB[RGB{N0f8}(0.725,0.475,0.773) RGB{N0f8}(0.773,0.522,0.82) … RGB{N0f8}(1.0,0.835,1.0) RGB{N0f8}(0.671,0.4,0.663); RGB

In [ ]:
#Comienzo Grayify

In [54]:
function resize_and_grayify(directory, im_name, width::Int64, height::Int64)
    resized_gray_img = Gray.(load(directory * "/" * im_name))
    try
        save("preprocessed_" * directory * "/" * im_name, resized_gray_img)
    catch e
        if isa(e, SystemError)
            mkdir("preprocessed_" * directory)
            save("preprocessed_" * directory * "/" * im_name, resized_gray_img)
        end
    end
end

resize_and_grayify (generic function with 1 method)

In [32]:
function process_images(directory, width::Int64, height::Int64)
    files_list = readdir(directory)
    map(x -> resize_and_grayify(directory, x, width, height),                               files_list)
end

process_images (generic function with 1 method)

In [33]:
n_resolution = 96

96

In [55]:
process_images("../input/histopathologic-cancer-detection/train", n_resolution, n_resolution)

Error encountered while load File{DataFormat{:CSV}, String}("../input/histopathologic-cancer-detection/train/_labels.csv").

Fatal error:


LoadError: ArgumentError: Package CSVFiles [5d742f6a-9f54-50ce-8119-2520741973ca] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.

Stacktrace:
  [1] [0m[1m_require[22m[0m[1m([22m[90mpkg[39m::[0mBase.PkgId[0m[1m)[22m
[90m    @ [39m[90mBase[39m [90m./[39m[90m[4mloading.jl:1306[24m[39m
  [2] [0m[1m_require_prelocked[22m[0m[1m([22m[90muuidkey[39m::[0mBase.PkgId[0m[1m)[22m
[90m    @ [39m[90mBase[39m [90m./[39m[90m[4mloading.jl:1200[24m[39m
  [3] [0m[1mmacro expansion[22m
[90m    @ [39m[90m./[39m[90m[4mlock.jl:223[24m[39m[90m [inlined][39m
  [4] [0m[1mrequire[22m[0m[1m([22m[90muuidkey[39m::[0mBase.PkgId[0m[1m)[22m
[90m    @ [39m[90mBase[39m [90m./[39m[90m[4mloading.jl:1195[24m[39m
  [5] [0m[1m#34[22m
[90m    @ [39m[90m~/.julia/packages/FileIO/aP78L/src/[39m[90m[4mloadsave.jl:203[24m[39m[90m [inlined][39m
  [6] [0m[1mlock[22m[0m[1m([

In [56]:
train_dir = readdir("preprocessed_/input/histopathologic-cancer-detection/train/")

137380-element Vector{String}:
 "00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif"
 "000020de2aa6193f4c160e398a8edea95b1da598.tif"
 "00004aab08381d25d315384d646f5ce413ea24b1.tif"
 "0000d563d5cfafc4e68acb7c9829258a298d9b6a.tif"
 "0000da768d06b879e5754c43e2298ce48726f722.tif"
 "0000f8a4da4c286eee5cf1b0d2ab82f979989f7b.tif"
 "00010f78ea8f878117500c445a658e5857f4e304.tif"
 "00011545a495817817c6943583b294c900a137b8.tif"
 "000126ec42770c7568204e2f6e07eb9a07d5e121.tif"
 "00014e39b5df5f80df56f18a0a049d1cc6de430a.tif"
 "00016a638ed77529609ac61479dc7ee99a6f3711.tif"
 "0001a2bc5d4aa55989f014bfad74a95ac3dfff54.tif"
 "0001b13d0d0f66d147a4167fca19fe22721556d4.tif"
 ⋮
 "9ffc4d94a9656c5fd276871cce329bdc741b5f83.tif"
 "9ffd11f6eac11a3dde57f2ae757199d37a818d7b.tif"
 "9ffd497b1ef39aaf72d0c57b000d9078e7778b66.tif"
 "9ffdb86461c02b2fc4d5a54911f9a45bd9787d36.tif"
 "9ffdce9f91102c39561e9cffa0c758bbb2685ffd.tif"
 "9ffe1057142e1c3f5bb8fe81d7f7e84ad7eba11e.tif"
 "9ffe7a2c28685e51609faa7b6116717db74c75ce.tif"
 "9fff

In [59]:

train_dir = first(train_dir, 1000)

1000-element Vector{String}:
 "00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif"
 "000020de2aa6193f4c160e398a8edea95b1da598.tif"
 "00004aab08381d25d315384d646f5ce413ea24b1.tif"
 "0000d563d5cfafc4e68acb7c9829258a298d9b6a.tif"
 "0000da768d06b879e5754c43e2298ce48726f722.tif"
 "0000f8a4da4c286eee5cf1b0d2ab82f979989f7b.tif"
 "00010f78ea8f878117500c445a658e5857f4e304.tif"
 "00011545a495817817c6943583b294c900a137b8.tif"
 "000126ec42770c7568204e2f6e07eb9a07d5e121.tif"
 "00014e39b5df5f80df56f18a0a049d1cc6de430a.tif"
 "00016a638ed77529609ac61479dc7ee99a6f3711.tif"
 "0001a2bc5d4aa55989f014bfad74a95ac3dfff54.tif"
 "0001b13d0d0f66d147a4167fca19fe22721556d4.tif"
 ⋮
 "012711068e1f75579088f135d4fe971de77f485a.tif"
 "012729816c718e8c29510fbec594aa2c6d444f35.tif"
 "012752b1b87c0080670a63f8ff548d008b01dcf7.tif"
 "0127b5a8952256267921f5915c0b82360939b19e.tif"
 "01292d179b3377a4cf6c2ae32b39fa553c89875d.tif"
 "01296d7d451384f8dbd517a2e623982c629bfaa2.tif"
 "012971d86e0b1df55080ce362657eab482dfff3e.tif"
 "0129e0

In [60]:

data = load.("preprocessed_/input/histopathologic-cancer-detection/train/" .* train_dir) 
#mostrar imágenes en escala de grises

1000-element Vector{Matrix{Gray{FixedPointNumbers.N0f8}}}:
 [Gray{N0f8}(0.808) Gray{N0f8}(0.749) … Gray{N0f8}(0.58) Gray{N0f8}(0.729); Gray{N0f8}(0.608) Gray{N0f8}(0.925) … Gray{N0f8}(0.804) Gray{N0f8}(0.706); … ; Gray{N0f8}(0.239) Gray{N0f8}(0.212) … Gray{N0f8}(0.639) Gray{N0f8}(0.616); Gray{N0f8}(0.169) Gray{N0f8}(0.275) … Gray{N0f8}(0.678) Gray{N0f8}(0.404)]
 [Gray{N0f8}(0.961) Gray{N0f8}(0.961) … Gray{N0f8}(0.886) Gray{N0f8}(0.847); Gray{N0f8}(0.961) Gray{N0f8}(0.961) … Gray{N0f8}(0.812) Gray{N0f8}(0.824); … ; Gray{N0f8}(0.98) Gray{N0f8}(0.882) … Gray{N0f8}(0.824) Gray{N0f8}(0.835); Gray{N0f8}(0.949) Gray{N0f8}(0.945) … Gray{N0f8}(0.765) Gray{N0f8}(0.725)]
 [Gray{N0f8}(0.808) Gray{N0f8}(0.392) … Gray{N0f8}(0.18) Gray{N0f8}(0.18); Gray{N0f8}(0.98) Gray{N0f8}(0.82) … Gray{N0f8}(0.047) Gray{N0f8}(0.541); … ; Gray{N0f8}(0.98) Gray{N0f8}(0.537) … Gray{N0f8}(0.137) Gray{N0f8}(0.31); Gray{N0f8}(0.392) Gray{N0f8}(0.839) … Gray{N0f8}(0.165) Gray{N0f8}(0.11)]
 [Gray{N0f8}(0.929) Gray{N0f8}(0

In [64]:
(x_train, y_train), (x_test, y_test) = splitobs(shuffleobs((data, df.label)), at = 0.7)
#explicar funciones

((Matrix{Gray{FixedPointNumbers.N0f8}}[[Gray{N0f8}(0.878) Gray{N0f8}(0.973) … Gray{N0f8}(0.914) Gray{N0f8}(0.902); Gray{N0f8}(0.773) Gray{N0f8}(0.882) … Gray{N0f8}(0.886) Gray{N0f8}(0.922); … ; Gray{N0f8}(0.925) Gray{N0f8}(0.396) … Gray{N0f8}(0.953) Gray{N0f8}(0.957); Gray{N0f8}(0.514) Gray{N0f8}(0.451) … Gray{N0f8}(0.949) Gray{N0f8}(0.949)], [Gray{N0f8}(0.894) Gray{N0f8}(0.925) … Gray{N0f8}(0.502) Gray{N0f8}(0.569); Gray{N0f8}(0.894) Gray{N0f8}(0.898) … Gray{N0f8}(0.341) Gray{N0f8}(0.345); … ; Gray{N0f8}(0.922) Gray{N0f8}(0.922) … Gray{N0f8}(0.922) Gray{N0f8}(0.922); Gray{N0f8}(0.922) Gray{N0f8}(0.922) … Gray{N0f8}(0.922) Gray{N0f8}(0.922)], [Gray{N0f8}(0.314) Gray{N0f8}(0.486) … Gray{N0f8}(0.102) Gray{N0f8}(0.114); Gray{N0f8}(0.341) Gray{N0f8}(0.565) … Gray{N0f8}(0.102) Gray{N0f8}(0.125); … ; Gray{N0f8}(0.431) Gray{N0f8}(0.376) … Gray{N0f8}(0.129) Gray{N0f8}(0.102); Gray{N0f8}(0.365) Gray{N0f8}(0.373) … Gray{N0f8}(0.141) Gray{N0f8}(0.11)], [Gray{N0f8}(0.467) Gray{N0f8}(0.514) … Gray{

In [71]:
# Las redes neuronales se pueden entrenar de manera más eficiente cuando los datos se transmiten en pequeños grupos
# o lotes de datos, en lugar de individualmente. 
# Este también es un procedimiento estándar cuando se trabaja con redes neuronales.
# La siguiente función agrupa los datos en mini batches para un posterior entrenamiento de la red.
# Luego creamos los minibatches, especificando un batchsize de 128. 
# El batchsize define el número de muestras que se propagarán a través de la red.
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:1)
    return (X_batch, Y_batch)
end
#explicar keras

make_minibatch (generic function with 1 method)

In [72]:
batchsize = 32
mb_idxs = partition(1:length(x_train), batchsize)
train_set = [make_minibatch(x_train, y_train, i) for i in mb_idxs]
test_set = make_minibatch(x_test, y_test, 1:length(y_test));

In [66]:
model = Chain(
        Conv((3, 3), 1=>32, pad=(1,1), relu),
        MaxPool((3,3)),
        Conv((3, 3), 32=>64, pad=(1,1), relu),
        MaxPool((3,3)),
        Conv((3, 3), 64=>128, pad=(1,1), relu),
        MaxPool((3,3)),
        flatten,
        Dense(15488, 2),
        softmax)

Chain(
  Conv((3, 3), 1 => 32, relu, pad=1),   # 320 parameters
  MaxPool((3, 3)),
  Conv((3, 3), 32 => 64, relu, pad=1),  # 18_496 parameters
  MaxPool((3, 3)),
  Conv((3, 3), 64 => 128, relu, pad=1),  # 73_856 parameters
  MaxPool((3, 3)),
  DataFrames.flatten,
  Dense(15488 => 2),                    # 30_978 parameters
  NNlib.softmax,
)                   # Total: 8 arrays, 123_650 parameters, 484.422 KiB.

In [73]:
train_loss = Float64[]
test_loss = Float64[]
acc = Float64[]
ps = Flux.params(model)
opt = ADAM()
L(x, y) = Flux.crossentropy(model(x), y)
L((x,y)) = Flux.crossentropy(model(x), y)
accuracy(x, y, f) = mean(Flux.onecold(f(x)) .== Flux.onecold(y))

function update_loss!()
    push!(train_loss, mean(L.(train_set)))
    push!(test_loss, mean(L(test_set)))
    push!(acc, accuracy(test_set..., model))
    @printf("train loss = %.2f, test loss = %.2f, accuracy = %.2f\n", train_loss[end], test_loss[end], acc[end])
end


update_loss! (generic function with 1 method)

In [74]:
@epochs 10 Flux.train!(L, ps, train_set, opt;
               cb = Flux.throttle(update_loss!, 8))

┌ Info: Epoch 1
└ @ Main /home/azul/.julia/packages/Flux/nJ0IB/src/optimise/train.jl:174


LoadError: MethodError: no method matching flatten(::Array{Float32, 4})
[0mClosest candidates are:
[0m  flatten([91m::AbstractDataFrame[39m, [91m::Union{Colon, Regex, AbstractString, Signed, Symbol, Unsigned, AbstractVector, All, Between, Cols, InvertedIndex}[39m) at ~/.julia/packages/DataFrames/KKiZW/src/abstractdataframe/abstractdataframe.jl:2541